# Tasks
The objective of this notebook is to use machine learning model to provide a ward national landcover in 2021. 

In [1]:
# import libraries
import os
import ee
import geemap

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AX4XfWjmrwXjDNR9lw1voWMPS_dZCWQtFMhh3p9jNL6FQSJFY93gBOzvAlc

Successfully saved authorization token.


In [3]:
!pwd

/Users/noe/Documents/PostDoc/Workshop/Day 3


## Select the ward of interest and load it raster

In [4]:
Map = geemap.Map(center=(-33.918861, 18.423300))
# Upload the shapefile of the ROI(SA)
w    = geemap.shp_to_ee('/Users/noe/Documents/PostDoc/Workshop/Day 3/SA4/ADM4_PCODE_ZA5950089.shp')
Map.addLayer(w, {}, 'ward_of_interest')
Map

Map(center=[-33.918861, 18.4233], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [30]:
def maskS2clouds(image):
    qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus(shadow), respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).bitwiseAnd(cirrusBitMask).eq(0)
    return image.updateMask(mask).divide(10000)

s2 = ee.ImageCollection('COPERNICUS/S2_SR').filterDate('2020-01-01', '2020-12-31').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).map(maskS2clouds)
# ///////////////// Use Median Band in parameters///////////////////////

median = s2.reduce(ee.Reducer.median())

# ////////////////Mosaic these two image collections///////////////////

mosaic_1 = ee.ImageCollection.fromImages([median]).mosaic()

# ////////////////////////Clip by assets or roi////////////////////////

sentinel_2_2020= mosaic_1.clip(w)

# /////////////// Display the clipped ImageCollection/////////////////

Map.addLayer(sentinel_2_2020, rgbVis, 'sentinel_roi_2020')
Map

Map(bottom=2464015.0, center=[-29.990846494016363, 30.910205841064457], controls=(WidgetControl(options=['posi…

In [27]:
def maskS2clouds(image):
    qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).bitwiseAnd(cirrusBitMask).eq(0)
    return image.updateMask(mask).divide(10000)

s2 = ee.ImageCollection('COPERNICUS/S2_SR').filterDate('2021-01-01', '2021-12-31').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).map(maskS2clouds)
# ///////////////// Use Median Band in parameters///////////////////////

median = s2.reduce(ee.Reducer.median())

# ////////////////Mosaic these two image collections///////////////////

mosaic_1 = ee.ImageCollection.fromImages([median]).mosaic()

# ////////////////////////Clip by assets or roi////////////////////////

sentinel_2_2021 = mosaic_1.clip(w)

# /////////////// Display the clipped ImageCollection/////////////////

Map.addLayer(sentinel_2_2021, rgbVis, 'sentinel_roi_2021')
Map

Map(bottom=2464015.0, center=[-29.990846494016363, 30.910205841064457], controls=(WidgetControl(options=['posi…

# Collecting the training data

In [28]:
# Load the SANLC 20202 for the roi
sanlcd2020 = ee.Image('users/noecareme/SA_NLC_2020_GEO').select('b1').clip(w)

In [29]:
# Generate the training dataset.

points = sanlcd2020.sample(**{
    'region': w,
    'scale': 20,
    'numPixels': 1500,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(points, {}, 'training', False)
Map

Map(bottom=2464015.0, center=[-29.990846494016363, 30.910205841064457], controls=(WidgetControl(options=['posi…

In [32]:
# Select the bands to use for classification.

bands = ['B1_median', 'B2_median', 'B3_median', 'B4_median', 'B5_median', 'B6_median', 'B7_median','B8_median','B8A_median','B9_median','B11_median','B12_median']

# This property of the table stores the land cover labels.
label = 'b1'

# Overlay the 1500 points from the SANLC2020 on the sentinel_2_2020 imagery.

sample = sentinel_2_2020.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 20
})

# Adds a column of deterministic pseudorandom numbers. 

sample = sample.randomColumn()

# Set the split parameter

split = 0.7
# Define the training and validation sets
training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

## Train the classifier

In [ ]:
classifier = ee.Classifier.smileRandomForest(5).train(training, label, bands)